In [23]:
import json
import math

import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.neighbors import KNeighborsClassifier
from tqdm import tqdm
from datetime import datetime
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
%load_ext jupyternotify


The jupyternotify extension is already loaded. To reload it, use:
  %reload_ext jupyternotify


In [ ]:
def ret(n, n_1):
    return (n_1 - n) / n


PREDICTIONS_TRAINING = 1000
PREDICTIONS_TESTING = 1000

# prices every 15 seconds
# df = pd.read_csv("prices.csv")
f = open("BTC_USDT-3m.json")
df = pd.DataFrame.from_records(json.load(f), columns=['timestamp', 'open', 'high', 'low', 'close', 'vol'])
df['ret_c'] = (df['close'] - df['close'].shift(1)) / df['close'].shift(1)
df['ret_o'] = (df['open'] - df['open'].shift(1)) / df['open'].shift(1)
df['ret_h'] = (df['high'] - df['high'].shift(1)) / df['high'].shift(1)
df['ret_l'] = (df['low'] - df['low'].shift(1)) / df['low'].shift(1)

df['ret'] = (df['open'].shift(-1) - df['open']) / df['open']

df['ret_s'] = df['ret'].apply(lambda x: np.sign(x))


In [25]:
df

,timestamp,open,high,low,close,vol,ret_c,ret_o,ret_h,ret_l,ret,ret_s
0,1661904000000,19813.03,19838.84,19811.14,19819.50,582.13461,NaN,NaN,NaN,NaN,0.000401,1.0
1,1661904180000,19820.97,19876.77,19797.94,19863.03,605.82699,0.002196,0.000401,0.001912,-0.000666,0.002223,1.0
2,1661904360000,19865.04,19910.78,19861.16,19904.99,578.33857,0.002112,0.002223,0.001711,0.003193,0.002052,1.0
3,1661904540000,19905.80,19910.18,19864.75,19873.44,607.35949,-0.001585,0.002052,-0.000030,0.000181,-0.001626,-1.0
4,1661904720000,19873.44,19892.60,19863.52,19883.66,354.87011,0.000514,-0.001626,-0.000883,-0.000062,0.000466,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
14668,1664544240000,19266.71,19269.34,19250.50,19254.16,962.51716,-0.000651,-0.000239,-0.000703,-0.000095,-0.000651,-1.0
14669,1664544420000,19254.16,19273.33,19243.61,19253.00,736.75091,-0.000060,-0.000651,0.000207,-0.000358,-0.000071,-1.0
14670,1664544600000,19252.79,19297.79,19237.19,19242.77,2040.90584,-0.000531,-0.000071,0.001269,-0.000334,-0.000446,-1.0
14671,1664544780000,19244.20,19357.70,19242.73,19298.84,2174.22268,0.002914,-0.000446,0.003105,0.000288,0.002839,1.0


In [26]:
# training
N_CANDLES = 6
NDIM = N_CANDLES * 4  # NDTR velas * 4 datos por velas
# K, NDTR, hit_rate


In [27]:
hit_rates_K = []
# K, NDTR, mean(ret)
rets_K = []
for K in tqdm(np.arange(1, 180, 3)):
    for NDTR in (np.arange(500, 5000, 500)):

        knn = KNeighborsClassifier(n_neighbors=K, weights="distance")

        xs = []
        y_train = []
        y_test = []
        y_test_ret = []
        y_predicted = []
        for j in np.arange(K + 1 + N_CANDLES, K + 1 + N_CANDLES + NDTR):
            small_df = df.iloc[j - N_CANDLES: j].copy(deep=True)[
                ['ret_c', 'ret_s', 'ret_o', 'ret_h', 'ret_l']].reset_index(
                drop=True)
            x = np.array(small_df[['ret_c',  'ret_h', 'ret_l', 'ret_h']].values).reshape(NDIM)
            y = df.iloc[j]['ret_s']
            xs.append(x)
            y_train.append(y)

        for j in np.arange(K + 1 + NDTR + N_CANDLES, K + 1 + NDTR + N_CANDLES + PREDICTIONS_TESTING):
            small_df = df.iloc[j - N_CANDLES: j].copy(deep=True)[
                ['ret_c', 'ret_s', 'ret_o', 'ret_h', 'ret_l']].reset_index(
                drop=True)
            x = np.array(small_df[['ret_c', 'ret_h', 'ret_l','ret_h']].values).reshape(NDIM)
            xs.append(x)
            y_test.append(df.iloc[j - 1]['ret_s'])
            y_test_ret.append(df.iloc[j - 1]['ret'])

        st_x = RobustScaler()
        xs_norm = st_x.fit_transform(xs)
        x_train = xs_norm[:NDTR]
        x_test = xs_norm[NDTR:]
        knn.fit(x_train, y_train)

        predictions = []
        actuals = []
        hits = 0
        rets = []

        for j in range(len(x_test)):
            prediction = -(knn.predict([x_test[j]])[0])
            actual = y_test[j]
            if actual == prediction:
                hits += 1
            if actual > 0:
                rets.append(y_test_ret[j])

        hit_rates_K.append([K, NDTR, hits / PREDICTIONS_TESTING])
        rets_K.append([K, NDTR, np.array(rets).mean() if len(rets) > 0 else 0])
%%notify


100%|██████████| 60/60 [58:24<00:00, 58.40s/it]
UsageError: Line magic function `%%notify` not found.


In [28]:
hit_rates_K = pd.DataFrame(hit_rates_K, columns=["K", "NDTR", "hit_rate"])
rets_K = pd.DataFrame(rets_K, columns=["K", "NDTR", "ret"])
print(hit_rates_K[hit_rates_K['hit_rate'] == hit_rates_K['hit_rate'].max()])
print(rets_K['ret'].max())

       K  NDTR  hit_rate
390  130  2000     0.631
0.0010158177597703237


In [29]:
hit_rates_K.to_csv("knn/first_run_knn_hr_1000testing_6candles_ochl_complement.csv")
rets_K.to_csv("knn/first_run_knn_ret_1000testing_6candles_ochl_complement.csv")